In [ ]:
import pandas as pd
import numpy as np
import sys
import os

np.set_printoptions(threshold=sys.maxsize)

In [ ]:
ID = "Unnamed: 0"

directory = os.path.dirname(os.getcwd())
df = pd.read_csv(directory + '\\datasets\\dna.csv', dtype={ID: object})

df = df.iloc[:, 0:5]
df = df.iloc[0:10, :]

print(df)

In [ ]:
poses = []

for sample in df[ID].values:
    sample = list(str(sample))
    base = np.array(['2']*20).astype(int)
    l = len(sample)
    base[:l] = sample
    poses.append(base)

data1 = df[df.columns[1:]] # df_wos = df_wo_samples

poses = pd.DataFrame(poses)
poses = poses.replace(2, np.NaN)
data2 = pd.concat([poses, data1], axis=1) # df_wos_with_poses

pacients_cols = df.columns[1:]
pacients_feachs = []
for pacient in pacients_cols:
    pacient_feachs = []
    for pos in range(20):
        pacient_feachs.append((data2[pos]*data2[pacient]).sum()/(data2[pacient][~data2[pos].isna()].sum()))
    pacients_feachs.append(pacient_feachs)
pacients_feachs = np.array(pacients_feachs)

pacients_feachs


In [ ]:
def count_pair(data, i, j):
    patterns = ["11", "01", "10", "00"]
    data_p = data
    data_p = data[~data[i].isna()]
    data_p = data_p[~data_p[j].isna()]
    i_col = data_p[i].astype(int).astype(str)
    j_col = data_p[j].astype(int).astype(str)
    pair_col = i_col+j_col
    res = []
    for p in patterns:
        res.append((pair_col == p).astype(int))
    return np.array(res)


pairs = []
for i in range(20):
    for j in range(20):
        if (i != j):
            pairs.append([i, j])

pacients_feachs2 = []
col_names = []
for pair in pairs:
    counts = count_pair(data2, pair[0], pair[1])
    pacient_feachs2 = []
    for pacient in pacients_cols:
        div = data2[pacient].sum()
        k = 0
        feach = []
        for p in counts:
            feach.append((p*data2[pacient]).sum()/div)
            col_names.append(
                f'{pacient}_{pair[0]}_{pair[1]}_{["11", "01", "10", "00"][k]}')
            k += 1
        pacient_feachs2.append(feach)
    pacients_feachs2.append(pacient_feachs2)
    
pacients_feachs2 = np.array(pacients_feachs2)

pacients_feachs2 = pacients_feachs2.transpose(1, 0, 2).reshape(pacients_feachs2.shape[1],
                                                               pacients_feachs2.shape[0] *
                                                               pacients_feachs2.shape[2],
                                                               )

pacients_feachs2


In [ ]:
all_feach = np.concatenate([pacients_feachs, pacients_feachs2], axis=-1)
logged_feach = np.log(all_feach)

labels = np.array(['p_cncr' in i for i in pacients_cols]).astype(int)

logged_feach
